In [1]:
import pandas as pd
from IPython.display import display
import warnings
import csv
import tabula

ModuleNotFoundError: No module named 'tabula'

In [4]:


warnings.filterwarnings("ignore")

file=open('C:/Users/min.gao/Desktop/BTBMW Int.txt', 'r')
content=file.readlines()
file.close()
dataframes=list()
flag=False
for line in content:
    if 'Range' in line and flag:
        dataframes.append(dataframe)
        dataframe=pd.DataFrame(columns=['Normalized'])
        index=0
    if 'Range' in line and not flag:
        dataframe=pd.DataFrame(columns=['Normalized'])
        flag=True
        index=0
    if '\t' in line and 'Range' not in line:
        segments=line.split('\t')
        number=pd.to_numeric(segments[2].strip())
        dataframe=dataframe.append(pd.DataFrame({'Normalized': [number]}, index=[index]))
        index+=1
dataframes.append(dataframe)
file_index=0
df_LIMS=pd.DataFrame(columns=['SAMPLE.SAMPLE_NUMBER',
                              'TEST.ANALYSIS',
                              'TEST.REPLICATE_COUNT',
                              'RESULT.NAME',
                              'RESULT.ENTRY',
                              'RESULT.UNITS',
                              'RESULT.Z_INSTRUMENT_RESULT1',
                              'RESULT.Z_INSTRUMENT_UNIT1',
                              'RESULT.Z_INSTRUMENT_RESULT2',
                              'RESULT.Z_INSTRUMENT_UNIT2',
                              'RESULT.ROUND',
                              'RESULT.PLACES',
                              'RESULT.MINIMUM',
                              'RESULT.Z_REP_SIG_FIGS',
                              'RESULT.FORMAT_CALCULATION',
                              'RESULT.REPORTABLE',
                              'RESULT.DISPLAYED',
                              'RESULT.X_REPORT_ORDER',
                              'RESULT.ORDER_NUMBER'])
######################################## update sample LIMS ID
IDs=["1368487"]
########################################
for df in dataframes:
    df.columns = ['Integral']
    df.insert(0, "Peak Assignment", ['hydrolysis region 1', 
                                     'BTBMW dimethyl at 3.5ppm', 
                                     'hydrolysis region 2 including DMA',
                                     'DMA at 2.2ppm',
                                     'BTBMW t-butyl at 1.4ppm',
                                     'BTBMW-1(Y1) at 1.43ppm',
                                     'BTBMW-2(Y2) at 1.27ppm',
                                     'hydrolysis region 3 including TBA', 
                                     'TBA at 0.98ppm'], True)
    # replace negative numbers with 0
    num = df._get_numeric_data()
    num[num < 0] = 0

    
    df.insert(2,"MW", [756.31, 414.23, 756.31, 45.08, 414.23, 442.280, 470.340, 756.31, 73.140], True)
    df.insert(3,'# of protons', [48, 12, 48, 6,18, 18, 18, 48, 9], True )
   
    #update hydrolysis region 2 and 3 values
    df.iloc[2,1] = df.iloc[2,1] - df.iloc[3,1]
    df.iloc[7,1] = df.iloc[7,1] - df.iloc[8,1]

    df['Mass'] = df['Integral']/df['# of protons']*df['MW']

    # replace negative numbers with 0
    num = df._get_numeric_data()
    num[num < 0] = 0

    BTBMW_Mass = (df['Mass'].values[1]+df['Mass'].values[4])/2
    df['Mass'].values[1] = BTBMW_Mass
    df['Mass'].values[4] = 'NaN'
    Hydro_Mass = df.iloc[0,4]+df.iloc[2,4]+df.iloc[7,4]
    df.iloc[0,4] = Hydro_Mass
    df.iloc[2,4] = 0
    df.iloc[7,4] = 0
    #calculate wt% of each component
    Total_Mass = df['Mass'].sum()
    df['wt%'] = df['Mass']/Total_Mass*100
    df=df.drop(df.index[[2,4,7]])
    df_report = df[['Peak Assignment', 'wt%']]        

    df_report.iloc[0, 0]='Impurity Z'
    df_report.iloc[1,0]='BTBMW'
    df_report.iloc[2,0] = 'DMA'
    df_report.iloc[3, 0]='BTBMW-1(Y1)'
    df_report.iloc[4, 0]='BTBMW-2(Y2)'
    df_report.iloc[5,0] = 'TBA'

    #rename columns
    df_report.rename(columns = {'Peak Assignment':'RESULT.NAME'},inplace = True)
    df_report.rename(columns = {'wt%': 'RESULT.ENTRY'},inplace = True)

    
    ID = [IDs[file_index] for i in range(6)]
    file_index+=1
    

    # adding lists as new column to dataframe df_report
    df_report['SAMPLE.SAMPLE_NUMBER'] = ID
    df_report['TEST.ANALYSIS'] = ['R0360' for i in range(6)] #repeat 'G0360' 7 times
    df_report['TEST.REPLICATE_COUNT'] = [1 for i in range(6)] 
    #rearrange columns
    df_report= df_report[['SAMPLE.SAMPLE_NUMBER','TEST.ANALYSIS','TEST.REPLICATE_COUNT','RESULT.NAME','RESULT.ENTRY']]
   
    df_report['RESULT.UNITS'] = ["PERCENT" for i in range (6)]
    df_report['RESULT.Z_INSTRUMENT_RESULT1'] = [" " for i in range (6)]  
    df_report['RESULT.Z_INSTRUMENT_UNIT1'] = [" " for i in range (6)]  
    df_report['RESULT.Z_INSTRUMENT_RESULT2'] = [" " for i in range (6)] 
    df_report['RESULT.Z_INSTRUMENT_UNIT2'] = [" " for i in range (6)] 
    df_report['RESULT.ROUND'] = [str('T') for i in range (6)] 
    df_report['RESULT.PLACES'] = ["6" for i in range (6)] 
    df_report['RESULT.MINIMUM'] = ["NA" for i in range (6)] 
    df_report['RESULT.Z_REP_SIG_FIGS'] = ["6" for i in range (6)] 
    df_report['RESULT.FORMAT_CALCULATION'] = ["COMPARE_TO_MINIMUM" for i in range (6)] 
    df_report['RESULT.REPORTABLE'] = ["T" for i in range (6)]  
    df_report['RESULT.DISPLAYED'] = ["T" for i in range (6)] 
    df_report['RESULT.X_REPORT_ORDER'] = ["1" for i in range (6)] 
    df_report['RESULT.ORDER_NUMBER'] = ['1' for i in range (6)] 
    df_LIMS=df_LIMS.append(df_report)
df_LIMS.to_csv("C:/Users/min.gao/Desktop/LIMS_BTBMW report.csv", encoding='utf-8', 
                     index=False, float_format='%.4f', quoting = csv.QUOTE_ALL)

    

display(df_LIMS)


,SAMPLE.SAMPLE_NUMBER,TEST.ANALYSIS,TEST.REPLICATE_COUNT,RESULT.NAME,RESULT.ENTRY,RESULT.UNITS,RESULT.Z_INSTRUMENT_RESULT1,RESULT.Z_INSTRUMENT_UNIT1,RESULT.Z_INSTRUMENT_RESULT2,RESULT.Z_INSTRUMENT_UNIT2,RESULT.ROUND,RESULT.PLACES,RESULT.MINIMUM,RESULT.Z_REP_SIG_FIGS,RESULT.FORMAT_CALCULATION,RESULT.REPORTABLE,RESULT.DISPLAYED,RESULT.X_REPORT_ORDER,RESULT.ORDER_NUMBER
0,1368487,R0360,1,Impurity Z,0.051941,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
1,1368487,R0360,1,BTBMW,99.347270,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
3,1368487,R0360,1,DMA,0.145975,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
5,1368487,R0360,1,BTBMW-1(Y1),0.138717,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
6,1368487,R0360,1,BTBMW-2(Y2),0.290229,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
8,1368487,R0360,1,TBA,0.025867,PERCENT,,,,,T,6,NA,6,COMPARE_TO_MINIMUM,T,T,1,1
